## Glyph error analysis

The `GlyphErrorBenchmark.cpp` creates a number of different rendered glyph textures into `src/rendered_glyph_textures` with different parameters (different code points, scales, original sizes etc.)

In this notebook I categorize these images and calculate the errors in each of them to figure out which method performs the best

In [9]:
# libraries
from enum import Enum
import os
from PIL import Image

In [10]:
# type definitions
class GlyphType(Enum):
    Original = 0
    Bitmap = 1
    SdfFromBitmap = 2
    SdfFromVector = 3
    Msdf = 4
    MsdfOriginal = 5

class Glyph:
    def __init__(self, glyphType, code_point, original_size, scale, img):
        self.glyphType = glyphType
        self.code_point = code_point
        self.original_size = original_size
        self.scale = scale
        self.img = img

    def __str__(self):
        return self.glyphType.name + ' ' + self.code_point + ' ' + str(self.original_size) + ' ' + str(self.scale) + ' ' + self.img.format

    def isComparable(self, other):
        return self.scale == other.scale and self.original_size == other.original_size and self.code_point == other.code_point

In [11]:
# load and filter all the glyphs
directory = '../src/rendered_glyph_textures'

truth_glyphs = []
glyphs = []

for filename in os.listdir(directory):
    if filename.endswith('.png'):
        info = filename.split('_')
        type = GlyphType(int(info[0][0]))
        code_point = info[1]
        original_size = int(info[2])
        scale = float(info[3])
        img = Image.open(os.path.join(directory, filename))

        if type == GlyphType.Original:
            truth_glyphs.append(Glyph(type, code_point, original_size, scale, img))
        else:
            glyphs.append(Glyph(type, code_point, original_size, scale, img))

In [22]:
def calculateGlyphError(truth, glyph):
    if not truth.isComparable(glyph):
        raise Exception('Glyphs are not comparable: ' + str(truth) + ' ' + str(glyph))


In [23]:
def findTruthForGlyph(glyph):
    for truth in truth_glyphs:
        if truth.isComparable(glyph):
            return truth
    return None

print('Number of glyphs: ' + str(len(glyphs)))
print('Number of truth glyphs: ' + str(len(truth_glyphs)))
# calculate all the glyph errors
for glyph in glyphs:
    truth = findTruthForGlyph(glyph)
    calculateGlyphError(truth, glyph)

Number of glyphs: 10
Number of truth glyphs: 2
truth size: (467, 586), glyph size: (457, 589), diff x: 10, diff y: -3
truth size: (326, 409), glyph size: (320, 409), diff x: 6, diff y: 0
truth size: (467, 586), glyph size: (467, 584), diff x: 0, diff y: 2
truth size: (326, 409), glyph size: (326, 409), diff x: 0, diff y: 0
truth size: (467, 586), glyph size: (466, 585), diff x: 1, diff y: 1
truth size: (326, 409), glyph size: (326, 409), diff x: 0, diff y: 0
truth size: (467, 586), glyph size: (466, 585), diff x: 1, diff y: 1
truth size: (326, 409), glyph size: (326, 409), diff x: 0, diff y: 0
truth size: (467, 586), glyph size: (461, 578), diff x: 6, diff y: 8
truth size: (326, 409), glyph size: (323, 405), diff x: 3, diff y: 4
